In [25]:
# Warning control
import warnings
import json
warnings.filterwarnings('ignore')
from crewai import Agent, Task, Crew
from crewai_tools import JSONSearchTool, WebsiteSearchTool
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

json_tool = JSONSearchTool()
web_tool = WebsiteSearchTool()

In [26]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_API_KEY"] = openai_api_key 

In [27]:
# Create the LLM configuration
llm_config = {
    "api_key": openai_api_key,
    "model": "gpt-3.5-turbo",
    "temperature": 0.1,
    "max_tokens": 500
}

In [28]:
import json
import os

# Define folder paths
input_folder = 'inputs'
output_folder = 'output'

# Read JSON files
with open(os.path.join(input_folder, 'persona.json'), 'r') as f:
    persona = json.load(f)
with open(os.path.join(input_folder, 'project.json'), 'r') as f:
    project = json.load(f)

In [52]:
inputs = [project, persona]

## Agents

In [29]:
user_persona_reader = Agent(
    role="Technical Profile Analyst",
    goal=f"Extract, normalize, and categorize all technical skills from user {persona} with high accuracy, ensuring no relevant skill is missed and maintaining consistent terminology.",
    backstory=f"""You are a seasoned technical recruiter who has reviewed thousands of developer profiles across various domains. 
    Your exceptional pattern recognition abilities allow you to identify technical skills even when they're described using non-standard 
    terminology or buried within project descriptions. You've developed a comprehensive mental taxonomy of the tech ecosystem 
    that enables you to categorize and normalize {persona}'s skills accurately. Your previous work at major tech companies involved building 
    skill assessment frameworks that accurately mapped candidate abilities to specific roles.""",
    verbose=True,
    allow_delegation=False,
    tools=[json_tool],  # Add tools here if needed for JSON parsing
    memory=True
)

In [30]:
project_requirement_reader = Agent(
    role="Technical Requirements Analyst",
    goal=f"Identify all explicit and implicit technical skills required for the {project}, ensuring comprehensive coverage of necessary technologies while filtering out irrelevant information.",
    backstory=f"""You are a senior solutions architect who has designed technical specifications for hundreds of enterprise 
    software projects. Your expertise spans the entire technology stack, from frontend frameworks to infrastructure components. 
    You have a talent for translating business requirements into precise technical specifications and can quickly identify 
    the full technology stack needed for the {project}. Your previous experience includes working as a technical lead who regularly 
    translated vague business requirements into detailed implementation plans.""",
    verbose=True,
    allow_delegation=False,
    tools=[json_tool],  # Add tools here if needed for JSON parsing
    memory=True
)

In [32]:
software_domain_expert = Agent(
    role="Full-Stack Technical Advisor",
    goal=f"Validate, enhance, and contextualize the required skills list with accurate dependencies, proficiency requirements, and project-specific applications for the {project}.",
    backstory="""You are a distinguished software architect with 15+ years of experience building enterprise systems across 
    multiple domains. You've mentored dozens of development teams and possess deep knowledge of how various technologies 
    interconnect within modern software ecosystems. Your experience includes working as a CTO for a software consultancy 
    where you regularly evaluated project requirements and assembled optimal technology stacks. You're known for your 
    comprehensive understanding of both established and emerging technologies.""",
    verbose=True,
    allow_delegation=True,
    tools=[web_tool],  # webscraper tool and anthropic API
    memory=True
)

In [34]:
gap_analyzer = Agent(
    role="Technical Skills Evaluator",
    goal=f"Accurately compare the output of {user_persona_reader} skills against {software_domain_expert} to identify specific gaps, categorize them meaningfully, and prioritize them according to project criticality.",
    backstory=f"""You are a respected technical educator with experience as both a developer and a learning consultant. 
    Your career has focused on assessing developer competencies identified by {user_persona_reader} and designing targeted learning paths identified by {software_domain_expert} to fill skill gaps. 
    You've worked with major online learning platforms to develop skill assessment tools that precisely identify knowledge gaps. 
    Your analytical approach allows you to efficiently compare skill profiles against requirements and determine the most 
    efficient path to close any gaps.""",
    verbose=True,
    allow_delegation=True,
    tools=[web_tool],  # webscraper tool and anthropic API
    memory=True
)

In [35]:
gap_report_formatter = Agent(
    role="Technical Gap Report Formatter",
    goal=f"Structure and format the {gap_analyzer}'s results into a detailed JSON report and save it to the output folder",
    backstory="""You are a technical documentation specialist with expertise in educational psychology, curriculum design, data structuring and JSON formatting. 
    Your strength lies in creating well-structured, machine-readable JSON outputs that follow standard formatting conventions 
    while maintaining human readability. You excel at organizing complex technical assessments into hierarchical JSON structures 
    that clearly represent relationships and priorities while ensuring all data is properly nested and formatted. Your talent for information organization and presentation helps developers quickly 
    understand what they need to learn and what is required for the project.""",
    verbose=True,
    allow_delegation=False,
    tools=[json_tool],  # Using the existing json_tool for JSON operations
    memory=True
)

In [36]:
gap_report_generator = Agent(
    role="Technical Learning Advisor",
    goal="Create a comprehensive, well-structured report that clearly communicates skill gaps and provides actionable learning recommendations.",
    backstory="""You are a technical documentation specialist with expertise in educational psychology and curriculum design. 
    Your career has been dedicated to transforming complex technical information into clear, actionable learning plans. 
    You've helped build several developer onboarding programs for tech companies, creating structured learning paths that 
    efficiently bridge knowledge gaps. Your talent for information organization and presentation helps developers quickly 
    understand what they need to learn and how to approach it.""",
    verbose=True,
    allow_delegation=False,
    tools=[],
    memory=True
)

## Tasks

In [37]:
extract_skills_task = Task(
    description="""
    Read the provided user {persona} JSON file and extract all technical skills mentioned in any field. Pay special attention to:
    1. Explicitly listed skills sections
    2. Project descriptions that mention technologies used
    3. Role titles that imply certain skills
    4. Educational background and certifications
    5. Any other fields that might indicate technical capabilities

    Don't miss implicit skills (e.g., if they mention "building REST APIs" they likely know HTTP protocols). 
    Extract both the skill name and any indicators of proficiency level (e.g., "basic Python", "5 years of JavaScript experience").
    
    The JSON file is located at: {persona}
    """,
    agent=user_persona_reader,
    expected_output="Raw list of all technical skills mentioned in the profile with associated proficiency indicators."
)

In [38]:
normalize_skills_task = Task(
    description="""
    Take the raw list of extracted skills and normalize them to standard industry terminology:
    1. Standardize naming conventions (e.g., "ReactJS" → "React", "MS SQL" → "SQL Server")
    2. Resolve abbreviations to their full names when appropriate
    3. Consolidate different versions of the same technology unless version differences are significant
    4. Standardize proficiency level descriptions to a consistent scale:
       - Beginner/Basic: Limited exposure or academic knowledge only
       - Intermediate: Working knowledge, can be productive with some guidance
       - Advanced: Deep knowledge, can work independently and solve complex problems
       - Expert: Complete mastery, can design systems and mentor others

    When normalizing, preserve any version-specific information that might be relevant.
    """,
    agent=user_persona_reader,
    expected_output="Normalized list of skills with standardized names and proficiency levels.",
    context=[extract_skills_task]
)

In [39]:
categorize_skills_task = Task(
    description="""
    Organize the normalized skills into a structured format with logical categories:
    1. Categorize by type:
       - Programming Languages
       - Frameworks & Libraries
       - Databases & Data Storage
       - DevOps & Infrastructure
       - Cloud Services & Platforms
       - Tools & Software
       - Methodologies & Practices
       - Domain-Specific Knowledge
    2. For each skill, include:
       - Standardized name
       - Normalized proficiency level
       - Years of experience (if available)
       - Context of use (if available)
    3. Format the output as a structured dictionary that clearly represents the user's current technical capabilities.
    
    Ensure the output is valid JSON format and properly structured for machine processing.
    """,
    agent=user_persona_reader,
    expected_output="Structured JSON dictionary of user skills organized by category with normalized terminology and proficiency levels.",
    context=[normalize_skills_task]
)

In [40]:
extract_explicit_requirements_task = Task(
    description="""
    Analyze the provided project requirements JSON file and extract all explicitly mentioned technical requirements:
    1. Identify all specifically named technologies, languages, frameworks, and tools
    2. Note any version requirements or constraints mentioned
    3. Extract environment or platform specifications
    4. Identify any explicitly mentioned technical skills or expertise
    5. Document any explicitly mentioned methodologies or practices

    Focus on requirements that are directly stated in the document, not inferred ones at this stage.
    
    The JSON file is located at: ${PROJECT_REQUIREMENTS_PATH}
    """,
    agent=project_requirement_reader,
    expected_output="List of all explicitly stated technical requirements with any specified versions or constraints."
)

In [41]:
infer_implicit_requirements_task = Task(
    description="""
    Based on the project description and explicit requirements, infer additional technical requirements that would be necessary:
    1. Identify technologies that are implied by explicitly mentioned ones (e.g., React implies JavaScript)
    2. Infer requirements based on described features or functionality
    3. Identify standard prerequisites for the explicitly mentioned technologies
    4. Consider infrastructure and deployment needs based on project scale and requirements
    5. Infer testing, security, or compliance requirements based on project type
    6. Document your reasoning for each inferred requirement to justify its inclusion

    Be thorough but avoid overspecifying - only include technologies that would reasonably be required.
    """,
    agent=project_requirement_reader,
    expected_output="List of inferred technical requirements with justification for each.",
    context=[extract_explicit_requirements_task]
)

In [42]:
normalize_requirements_task = Task(
    description="""
    Take the combined explicit and implicit requirements and organize them:
    1. Normalize terminology to industry standards
    2. Categorize requirements by:
       - Layer (Frontend, Backend, Database, DevOps, etc.)
       - Type (Languages, Frameworks, Tools, Platforms, etc.)
    3. Indicate which requirements are:
       - Core/Critical: Essential to project success
       - Important: Strongly preferred but alternatives might work
       - Optional: Nice to have but not essential
    4. Note any dependencies between requirements
    5. Format as a comprehensive, structured dictionary of project technical requirements
    
    Ensure the output is valid JSON format and properly structured for machine processing.
    """,
    agent=project_requirement_reader,
    expected_output="Structured dictionary of normalized project requirements categorized by type and criticality.",
    context=[extract_explicit_requirements_task, infer_implicit_requirements_task]
)

In [43]:
validate_requirements_task = Task(
    description="""
    Review the project requirement reader's output and validate the technical requirements:
    1. Verify that all necessary technologies for the project are included
    2. Check for any inappropriate or unnecessary technologies
    3. Identify any technological conflicts or incompatibilities
    4. Ensure the technology stack is coherent and follows industry best practices
    5. Verify that all major architectural components are covered
    6. Check that the proposed technologies are appropriate for the project's scale and complexity
    7. Validate that all critical security and performance needs are addressed
    8. Add any missing technologies that would be essential for implementation

    Provide justification for any changes or additions you make.
    """,
    agent=software_domain_expert,
    expected_output="Validated and potentially enhanced list of technical requirements with justifications for any changes.",
    context=[normalize_requirements_task]
)

In [44]:
define_dependencies_task = Task(
    description="""
    For each technical requirement:
    1. Identify prerequisite skills and technological dependencies
    2. Specify the minimum proficiency level required:
       - Beginner: Basic understanding is sufficient
       - Intermediate: Working knowledge required
       - Advanced: Deep expertise needed
    3. Create a dependency graph showing relationships between skills
    4. Indicate which skills build upon others
    5. Specify whether skills need to be used in combination
    6. Define the breadth vs. depth of knowledge required
    7. Note any specific techniques or patterns within each technology that are particularly relevant

    Be specific about exactly what aspects of each technology are needed.
    """,
    agent=software_domain_expert,
    expected_output="Enhanced requirements list with dependency relationships and required proficiency levels for each skill.",
    context=[validate_requirements_task]
)

In [45]:
contextualize_requirements_task = Task(
    description="""
    For each validated technical requirement:
    1. Explain how it specifically applies to this project
    2. Describe the role it plays in the overall architecture
    3. Identify the specific features or functions it enables
    4. Note any project-specific implementations or patterns that would be used
    5. Explain why this particular technology is appropriate for this use case
    6. Identify any alternatives that could potentially be substituted
    7. Rate its criticality to project success (high/medium/low)
    8. Estimate the relative complexity of its implementation in this context

    Provide practical context that helps understand how each technology would be used in the project.
    
    Ensure the output is valid JSON format and properly structured for machine processing.
    """,
    agent=software_domain_expert,
    expected_output="Contextualized requirements document that explains how each technology applies to the specific project with criticality ratings.",
    context=[define_dependencies_task]
)

In [46]:
compare_skills_task = Task(
    description="""
    Compare the user's current skills (from User Persona Reader) against the required skills (from Software Domain Expert):
    1. Match user skills to project requirements using normalized terminology
    2. Account for different naming conventions or variations
    3. Consider proficiency levels in your comparison
    4. Note where user has skills that exceed requirements
    5. Identify skills present in requirements but absent from user profile
    6. Detect skills where user has some experience but not at required level
    7. Consider related skills that might partially transfer
    8. Create a comprehensive mapping between user skills and project requirements

    Be thorough in your comparison and document your rationale.
    """,
    agent=gap_analyzer,
    expected_output="Detailed comparison showing how each required skill maps to the user's existing skillset.",
    context=[categorize_skills_task, contextualize_requirements_task]
)

In [47]:
categorize_gaps_task = Task(
    description="""
    For each required skill, categorize the gap and assign priority:

    1. Gap Categories:
       - "Missing": User has no experience with this technology
       - "Needs Improvement": User has some experience but below required level
       - "Sufficient": User meets or exceeds the required proficiency

    2. For each gap, assign priority based on:
       - Criticality to project success
       - Difficulty to learn or improve
       - Dependency relationships with other skills
       - Time sensitivity within project timeline
       
    3. For "Needs Improvement" gaps, specify:
       - Current estimated proficiency level
       - Required proficiency level
       - Specific aspects that need enhancement

    Justify each categorization and priority assignment with clear reasoning.
    """,
    agent=gap_analyzer,
    expected_output="Categorized skill gaps with priorities and detailed assessment of improvement needs.",
    context=[compare_skills_task]
)

In [48]:
structure_gap_analysis_task = Task(
    description="""
    Organize the gap assessment into a structured format:
    1. Create a JSON structure with three main sections:
       - "missing": Skills completely absent from user profile
       - "needs_improvement": Skills requiring enhancement
       - "sufficient": Skills where user meets requirements
    2. For each skill, include:
       - Priority level (high/medium/low)
       - Category/type of skill
       - Dependency relationships
       - Proficiency gap details (for "needs_improvement")
    3. Include contextual notes about how the skill applies to the project
    4. Ensure the structure is consistent and well-organized
    5. Make the output programmatically parsable while remaining human-readable
    
    Ensure the output is valid JSON format and properly structured for machine processing.
    """,
    agent=gap_analyzer,
    expected_output="Structured JSON document with comprehensive gap analysis organized by category.",
    context=[categorize_gaps_task]
)

In [49]:
organize_gap_info_task = Task(
    description="""
    Take the gap analysis from the Gap Analyzer and organize it into a JSON structure:
    1. Create a hierarchical organization with main categories:
       - Missing skills (sorted by priority)
       - Skills needing improvement (sorted by priority)
       - Sufficient skills (grouped by domain)
    2. Within each category, sort skills by priority and dependency relationships"
    3. Include detailed proficiency level comparisons and improvement suggestions
    4. Ensure the structure is consistent and easy to navigate
    5. Ensure all critical information is preserved and clearly presented
    
    """,
    agent=gap_analyzer,
    expected_output="Structured JSON document with comprehensive gap analysis organized by category.",
    context=[structure_gap_analysis_task]
)

In [50]:
generate_report_task = Task(
    description="""
    1. Take the gap analysis from the Gap Analyzer.
    2. Organize the results into a well-structured JSON report.
    3. Group skills by their gap category (missing, needs improvement, sufficient).
    4. Within each category, organize by skill domain/type.
    5. For missing and improvement skills, add brief learning recommendations.
    6. Ensure the JSON is properly formatted and readable.
    7. Write the final report to a JSON file at the specified output path.
    """,
    agent=gap_report_generator,
    expected_output="A well-structured JSON file containing the skill gap analysis organized by categories with learning recommendations.",
    dependencies=[organize_gap_info_task]
)

In [51]:
# Set up the crew (replace existing crew setup)
crew = Crew(
    agents=[user_persona_reader, project_requirement_reader, software_domain_expert, gap_analyzer, gap_report_formatter, gap_report_generator],
    tasks=[categorize_skills_task, contextualize_requirements_task, compare_skills_task, categorize_gaps_task, structure_gap_analysis_task, organize_gap_info_task, generate_report_task],
    llm_config=llm_config,
    verbose=True)

In [53]:
# Kickoff the crew with inputs
result = crew.kickoff()

# Print the result
print(result)

# Agent: Technical Profile Analyst
## Task: 
    Organize the normalized skills into a structured format with logical categories:
    1. Categorize by type:
       - Programming Languages
       - Frameworks & Libraries
       - Databases & Data Storage
       - DevOps & Infrastructure
       - Cloud Services & Platforms
       - Tools & Software
       - Methodologies & Practices
       - Domain-Specific Knowledge
    2. For each skill, include:
       - Standardized name
       - Normalized proficiency level
       - Years of experience (if available)
       - Context of use (if available)
    3. Format the output as a structured dictionary that clearly represents the user's current technical capabilities.
    
    Ensure the output is valid JSON format and properly structured for machine processing.
    


Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it]
2025-03-16 17:53:49,946 - 31708 - local_persistent_hnsw.py-local_persistent_hnsw:423 - WARNING: Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1




# Agent: Technical Profile Analyst
## Thought: I need to extract and normalize the skills from the provided JSON, organizing them into the required categories and formatting them correctly.
## Using tool: Search a JSON's content
## Tool Input: 
"{\"search_query\": \"skills\", \"json_path\": \"Skills\"}"
## Tool Output: 
Relevant Content:
Skills


Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]2025-03-16 17:53:52,320 - 31708 - sqlite.py-sqlite:297 - WARNING: Insert of existing embedding ID: default-app-id--8ae5832c91b188ae615d33ef0079f1877c9589dd43f2ba38f93684becf1f7daa
2025-03-16 17:53:52,322 - 31708 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--8ae5832c91b188ae615d33ef0079f1877c9589dd43f2ba38f93684becf1f7daa
Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.05it/s]
2025-03-16 17:53:52,743 - 31708 - local_persistent_hnsw.py-local_persistent_hnsw:423 - WARNING: Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1




# Agent: Technical Profile Analyst
## Using tool: Search a JSON's content
## Tool Input: 
"{\"search_query\": \"DataScience\", \"json_path\": \"Skills\"}"
## Tool Output: 
Relevant Content:
Skills


# Agent: Technical Profile Analyst
## Final Answer: 
{
    "skills": {
        "Programming Languages": [
            {
                "standardized_name": "Python",
                "normalized_proficiency_level": "85",
                "years_of_experience": "2",
                "context_of_use": "Data science and software development tasks."
            }
        ],
        "Frameworks & Libraries": [
            {
                "standardized_name": "Pandas",
                "normalized_proficiency_level": "85",
                "years_of_experience": "2",
                "context_of_use": "Data manipulation and analysis."
            },
            {
                "standardized_name": "Power BI",
                "normalized_proficiency_level": "85",
                "years_of_experie

...



# Agent: Technical Gap Report Formatter
## Using tool: Search a JSON's content
## Tool Input: 
"{\"search_query\": \"Complete assessment of skills\", \"json_path\": \"$.Skills\"}"
## Tool Output: 
Relevant Content:
Skills

$.Skills

$.EducationalBackground


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Search a JSON's content
Tool Arguments: {'search_query': {'description': "Mandatory search query you want to use to search the JSON's content", 'type': 'str'}, 'json_path': {'description': 'Mandatory json path you want to search', 'type': 'str'}}
Tool Description: A tool that can be used to semantic search a query from a JSON's content.

IMPORTANT: Use the following format in your response:

```
Thought: you should always think about what to do
Action: the action to take, only one name of [Search a JSON's content], just the name, exactly as it's written.
Action Input: the input to the action, just a simple JSON object

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]2025-03-16 17:57:10,502 - 31708 - sqlite.py-sqlite:297 - WARNING: Insert of existing embedding ID: default-app-id--f9c72cf840800b37346e872ce74810c9fb73d0439b144849ea7a7deccb1dc3af
2025-03-16 17:57:10,505 - 31708 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--f9c72cf840800b37346e872ce74810c9fb73d0439b144849ea7a7deccb1dc3af
Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.12it/s]




# Agent: Technical Gap Report Formatter
## Using tool: Search a JSON's content
## Tool Input: 
"{\"search_query\": \"skills categories and recommendations\", \"json_path\": \"$.Skills\"}"
## Tool Output: 
Relevant Content:
Skills

$.Skills

$.EducationalBackground


Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]2025-03-16 17:57:13,067 - 31708 - sqlite.py-sqlite:297 - WARNING: Insert of existing embedding ID: default-app-id--b4ebb269410d7d032d0e9a1cb29c2ad0f965ca74223e114d0c9712f9822bcde5
2025-03-16 17:57:13,070 - 31708 - local_persistent_hnsw.py-local_persistent_hnsw:339 - WARNING: Add of existing embedding ID: default-app-id--b4ebb269410d7d032d0e9a1cb29c2ad0f965ca74223e114d0c9712f9822bcde5
Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.21it/s]




# Agent: Technical Gap Report Formatter
## Using tool: Search a JSON's content
## Tool Input: 
"{\"search_query\": \"Final assessment structure\", \"json_path\": \"$\"}"
## Tool Output: 
Relevant Content:
$.EducationalBackground

Skills

$.gapAnalysis


# Agent: Technical Gap Report Formatter
## Final Answer: 
```json
{
  "gapAnalysis": {
    "missingSkills": [
      {
        "name": "Cloud Computing",
        "priority": "high",
        "proficiencyLevel": "N/A",
        "improvementSuggestions": [
          "Take online courses in AWS and Azure.",
          "Participate in cloud computing projects or internships."
        ]
      }
    ],
    "improvementNeededSkills": [
      {
        "name": "Data Science",
        "currentProficiency": 85,
        "industryAverageProficiency": 90,
        "priority": "medium",
        "improvementSuggestions": [
          "Engage in more hands-on data science projects.",
          "Contribute to open-source data science initiatives."
        ]

In [54]:
import json

# Get the JSON content from the result
json_content = result.raw

# Clean up the JSON string by removing markdown code blocks
json_content = json_content.replace('```json', '').replace('```', '').strip()

try:
    # Parse and save as JSON file
    with open('out.json', 'w') as f:
        json.dump(json.loads(json_content), f, indent=2)
    print("JSON file saved successfully!")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    print("Raw content:", json_content)

JSON file saved successfully!
